In [2]:
import os
for dirname, _, filenames in os.walk("C:/Users/govar/Downloads/archive"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/govar/Downloads/archive\movies.csv
C:/Users/govar/Downloads/archive\ratings.csv


In [3]:
!pip install sentence_transformers

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 1.5 MB/s eta 0:00:07
   --- ------------------------------------ 0.8/10.0 MB 1.5 MB/s eta 0:00:07
   ----- ---------------------------------- 1.3/10.0 MB 1.7 MB/s eta 0:00:06
   ------ --------------------------------- 1.6/10.0 MB 1.8 MB/s eta 0:00:05
   -------- ------------------------------- 2.1/10.0 MB 1.9 MB/s eta 0:00:05
   ---------- ----------------------------- 2.6/10.0 MB 2.0 MB/s eta 0:00:04
   ------------- -------------------------- 3.4/10.0 MB 2.2 MB/s eta 0:00:03
   --------------- ------------------------ 3.9/10.0 MB 2.3 MB/s eta 0:00:03
   ------------------ --------------------- 4.7/10.0 MB 2.4 MB/s eta 0:00:03
   ---------------------- ----------------- 5.8/10.0 MB 2.7 MB/s eta 0:00:02
   --------------------------- ------------ 6.8/10.0 MB 2.9 MB/s eta 0:00:02
   ----------

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
movies = pd.read_csv('C:/Users/govar/Downloads/archive/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [6]:
movies['description'] = 'Title: ' + movies['title'] + ', Genre: ' + movies['genres']
movies

,movieId,title,genres,description
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Title: Toy Story (1995), Genre: Adventure|Anim..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"Title: Jumanji (1995), Genre: Adventure|Childr..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"Title: Grumpier Old Men (1995), Genre: Comedy|..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Title: Waiting to Exhale (1995), Genre: Comedy..."
4,5,Father of the Bride Part II (1995),Comedy,"Title: Father of the Bride Part II (1995), Gen..."
...,...,...,...,...
62418,209157,We (2018),Drama,"Title: We (2018), Genre: Drama"
62419,209159,Window of the Soul (2001),Documentary,"Title: Window of the Soul (2001), Genre: Docum..."
62420,209163,Bad Poems (2018),Comedy|Drama,"Title: Bad Poems (2018), Genre: Comedy|Drama"
62421,209169,A Girl Thing (2001),(no genres listed),"Title: A Girl Thing (2001), Genre: (no genres ..."


In [15]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [16]:
tokenizer

BertTokenizerFast(name_or_path='sentence-transformers/all-MiniLM-L6-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [14]:

def get_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding = True, truncation = True, return_tensors = 'pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p = 2, dim = 1)
    return sentence_embeddings

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
movies_embeddings = model.encode(movies['description'].tolist())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def get_recommendations(query, embeddings, df, top_n = 5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [10]:
query = "Animated movie about bears that I can watch with kids"
recommendations = get_recommendations(query, movies_embeddings, movies)
recommendations[['title', 'genres']]

,title,genres
38301,The Little Bear Movie (2001),Animation|Children
5431,Care Bears Movie II: A New Generation (1986),Animation|Children
5430,"Care Bears Movie, The (1985)",Animation|Children|Fantasy
38230,The Little Polar Bear (2001),Animation|Children
37961,Bear Story (2014),Animation


In [11]:
query = "Movies falling under the comedy and horror genres"
recommendations = get_recommendations(query, movies_embeddings, movies)
recommendations[['title', 'genres']]

,title,genres
46122,Range 15 (2016),Comedy|Horror
32261,"Visit, The (2015)",Comedy|Horror
41423,Scare Campaign (2016),Comedy|Horror
7399,Versus (2000),Action|Comedy|Fantasy|Horror
31265,Stalled (2013),Comedy|Horror


In [12]:
query = "Funny hindi comedies to watch"
recommendations = get_recommendations(query, movies_embeddings, movies)
recommendations[['title', 'genres']]

,title,genres
48642,Hindi Medium (2017),Comedy|Drama
5480,Indian Summer (1993),Comedy|Drama
42734,Delhi in a Day (2012),Comedy|Drama
53106,Oru Indian Pranayakadha (2013),Comedy|Romance
14497,India (Indien) (1993),Comedy|Drama


In [13]:
query = "Romantic comedies released in the 1990s"
recommendations = get_recommendations(query, movies_embeddings, movies)
recommendations[['title', 'genres']]

,title,genres
15552,"Romantics, The (2010)",Comedy|Drama|Romance
7992,Funny About Love (1990),Comedy|Romance
55735,The New Romantic (2018),Comedy|Drama
184,Nine Months (1995),Comedy|Romance
7772,Book of Love (1990),Comedy|Romance
